In [0]:
dbutils.library.restartPython()

In [0]:

%sh

pip install azure-identity
pip install azure-monitor-query
pip install azure-mgmt-resourcegraph
pip install azure-mgmt-subscription
pip install netaddr

  Obtaining dependency information for azure-identity from https://files.pythonhosted.org/packages/3d/9f/1f9f3ef4f49729ee207a712a5971a9ca747f2ca47d9cbf13cf6953e3478a/azure_identity-1.21.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.8 MB/s eta 0:00:00
  Obtaining dependency information for azure-core>=1.31.0 from https://files.pythonhosted.org/packages/84/9e/5c87b49f65bb16571599bc789857d0ded2f53014d3392bc88a5d1f3ad779/azure_core-1.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.5 MB/s eta 0:00:00
  Obtaining dependency information for msal>=1.30.0 from https://files.pythonhosted.org/packages/04/bf/81516b9aac7fd867709984d08eb4db1d2e3fe1df795c8e442cde9b568962/msal-1.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for msal-extensions>=1.2.0 from https://files.pythonhosted.org/packages/5e/75/bd9b7bb966668920f06b200e84454c8f3566b102183bc55c5473d96cb2b9/msal_extensions-1.3.1-py3-none-any.w


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for azure-monitor-query from https://files.pythonhosted.org/packages/cb/71/bdf03aef699d9a17d80bcaa8255c20e596ed591daa8a42a21854ddc4277d/azure_monitor_query-1.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 10.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for azure-mgmt-resourcegraph from https://files.pythonhosted.org/packages/bd/16/63c37bffdce5082c9997ad7783921b02ed534b5971e5bdbd1ae617a5b2e3/azure_mgmt_resourcegraph-8.0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for msrest>=0.6.21 from https://files.pythonhosted.org/packages/15/cf/f2966a2638144491f8696c27320d5219f48a072715075d168b31d3237720/msrest-0.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for azure-common~=1.1 from https://files.pythonhosted.org/packages/62/55/7f118b9c1b23ec15ca05d15a578d8207aa1706bc6f7c87218efffbbf875d/azure_common-1.1.28-py2.py3-none-any.whl.metadata
  Obtaining dependency information for azure-mgmt-core<2.0.0,>=1.2.0 from https://files.pythonhosted.org/packages/ab/2d/762b027cfd58b1b2c9b5b60d112615bd04bc33ef85dac55d2ee739641054/azure_mgmt_core-1.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for requests-oauthlib>=0.5.0 from https://files.pythonhosted.org/packages/3


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for azure-mgmt-subscription from https://files.pythonhosted.org/packages/70/12/e6de2021c4689f857386670ba0b6d3c4025d4209e45df7dd7cdabe9a4ac1/azure_mgmt_subscription-3.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 2.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


  Obtaining dependency information for netaddr from https://files.pythonhosted.org/packages/12/cc/f4fe2c7ce68b92cbf5b2d379ca366e1edae38cccaad00f69f529b460c3ef/netaddr-1.3.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip



### doc reference 

https://learn.microsoft.com/en-us/azure/network-watcher/traffic-analytics-schema?tabs=vnet#traffic-analytics-schema

In [0]:
vnet_logs = '''

let traffic_analytics_pip_table = NTAIpDetails
        | project ['PublicIP']=Ip, ['PIP_PublicIpDetails']=PublicIpDetails, ['PIP_ThreatType']=ThreatType, ['PIP_Location']=Location, ['PIP_Url']=Url;

NTANetAnalytics

| where FlowType != '' and SubType == 'FlowLog'

// filter grafana $__timeFrom, $__timeTo
//| where TimeGenerated between(startofday(datetime(2025-4-27)) .. endofday(datetime(2025-5-1)))
//| where TimeGenerated >= ago(1d)

| take 6000

| extend SrcPIP = substring(SrcPublicIps, 0, indexof(SrcPublicIps, "|"))
| extend DestPIP = substring(DestPublicIps, 0, indexof(DestPublicIps, "|"))

| extend SrcSubnetSplitted =  split(SrcSubnet, '/')
| extend SrcRG = tostring(SrcSubnetSplitted[0])
| extend SrcVNet = tostring(SrcSubnetSplitted[1])
| extend SrcSubnetName = tostring(SrcSubnetSplitted[2])

| extend DestSubnetSplitted =  split(DestSubnet, '/')
| extend DestRG = tostring(DestSubnetSplitted[0])
| extend DestVNet = tostring(DestSubnetSplitted[1])
| extend DestSubnetName = tostring(DestSubnetSplitted[2])



// resolve Azure Public IP from NTAIpDetails table
| join kind=leftouter
(
    traffic_analytics_pip_table
| project ['SrcPIP_Ip']=PublicIP, ['SrcPIP_PublicIpDetails']=PIP_PublicIpDetails, ['SrcPIP_ThreatType']=PIP_ThreatType, ['SrcPIP_Location']=PIP_Location, ['SrcPIP_Url']=PIP_Url
)
on $left.SrcPIP == $right.SrcPIP_Ip

| join kind=leftouter
    (
        traffic_analytics_pip_table
        | project ['DestPIP_Ip']=PublicIP, ['DestPIP_PublicIpDetails']=PIP_PublicIpDetails, ['DestPIP_ThreatType']=PIP_ThreatType, ['DestPIP_Location']=PIP_Location, ['DestPIP_Url']=PIP_Url
    )
on $left.DestPIP == $right.DestPIP_Ip

// set SrcPIP_Ip if exist
| extend SrcIp = iif(SrcPIP_Ip != '', SrcPIP_Ip, SrcIp)
// set DestPIP_Ip if exist
| extend DestIp = strcat(iif(DestPIP_Ip != '', DestPIP_Ip, DestIp), ':', DestPort)

//check "location" of src node
// | extend SrcNodeInOrOutVNet =   iif(SrcIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
//                                   iif(FlowType == 'AzurePublic' and SrcPublicIps != '', 'NodeOutsideVNet_PaaS',
//                                     iif(FlowType == 'ExternalPublic' and SrcPublicIps != '', 'NodeOutsideAzure',
//                                       //adversary from outside of Azure
//                                       iif (FlowType =='MaliciousFlow' and FlowDirection == 'Inbound' and SrcIp != '', 'NodeMalicious',
//                                         //adversary from inside of Azure
//                                         iif(FlowType =='MaliciousFlow' and FlowDirection == 'Outbound', 'NodeInsideVNet', 'NodeUnknown')
//                                 )
//                               )
//                             )
//                           )

//check "location" of dest node
// | extend DestNodeInOrOutVNet = iif(DestIp != '' and FlowType != 'MaliciousFlow', 'NodeInsideVNet',
//                             iif(FlowType in ('AzurePublic') and DestPublicIps != '', 'NodeOutsideVNet_PaaS',
//                                 iif(FlowType == 'ExternalPublic' and DestPublicIps != '', 'NodeOutsideAzure',
//                                     //adversary from outside of Azure
//                                     iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Inbound' and DestIp != '', 'NodeInsideVNet',
//                                         //adversary from inside of Azure
//                                         iif(FlowType == 'MaliciousFlow' and FlowDirection== 'Outbound', 'NodeMalicious', 'NodeUnknown')
//                                 )
//                               )
//                             )
//                           )


// resolve SrcName as private endpoint name if traffic is private endpoint
// *note: private endpoint is Source and not destination
| extend PrivateEndpointName = split(PrivateEndpointResourceId, '/')[-1]
| extend PrivateLinkName = split(PrivateEndpointResourceId, '/')[-2]

| extend SrcName = iif(SrcApplicationGateway != '', SrcApplicationGateway,
                     iif(SrcLoadBalancer != '', SrcLoadBalancer,
                        iif(SrcPIP_PublicIpDetails != '', SrcPIP_PublicIpDetails,
                          iif(SrcSubnetName == 'azurebastionsubnet', 'bastion-vm',
                           iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateEndpointName,
                             iif(SrcNic startswith 'unknown', strcat('managed vm in ', iif(SrcSubnetName has 'subnet', SrcSubnetName, strcat(SrcSubnetName, ' subnet'))),
                               iif(SrcVm != '', SrcVm, '-' )))))))

| extend SrcName = iif(indexof(SrcName, '/',0) > 0, split(SrcName, '/')[-1], SrcName)


| extend DestName = iif(DestApplicationGateway != '', DestApplicationGateway,
                        iif(DestLoadBalancer != '', DestLoadBalancer,
                            iif(DestPIP_PublicIpDetails != '', DestPIP_PublicIpDetails,
                               iif(DestSubnetName == 'azurebastionsubnet', 'bastion-vm',
                                iif(PrivateEndpointResourceId != '' and PrivateLinkResourceId != '',  PrivateLinkName,
                                  iif(DestNic startswith 'unknown', strcat('managed vm in ', iif(DestSubnetName has 'subnet', DestSubnetName, strcat(DestSubnetName, ' subnet'))),
                                    iif(DestVm != '',  DestVm, '' )))))))


| extend DestName = iif(indexof(DestName, '/',0) > 0, split(DestName, '/')[-1], DestName)


| extend protocol = tolower(strcat(L4Protocol,  iif(L7Protocol != 'Unknown', strcat(':', L7Protocol), '')))


| summarize BytesSrcToDest = max(BytesSrcToDest), BytesDestToSrc = max(BytesDestToSrc)
            by  FlowType, FlowDirection, ConnectionType, protocol,
                    SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName,
                    DestSubscription, DestRG, DestVNet, DestSubnetName, DestName,
                    SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
                    DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url


| extend SrcToDestDataSize = format_bytes(BytesSrcToDest, 2)
| extend DestToSrcDataSize = format_bytes(BytesDestToSrc, 2)

| distinct FlowType, FlowDirection, ConnectionType, Protocol=protocol,
        SrcSubscription, SrcRG, SrcVNet, SrcSubnetName , SrcIp, SrcName, SrcToDestDataSize,
        DestSubscription, DestRG, DestVNet, DestSubnetName, DestName, DestToSrcDataSize,
        SrcPIP_ThreatType, SrcPIP_Location, SrcPIP_Url,
        DestIp, DestPIP_ThreatType, DestPIP_Location, DestPIP_Url

'''


### main df - vnet flow logs, drop duplicates and fill null

In [0]:
from azure.monitor.query import LogsQueryClient
from azure.monitor.query._models import LogsQueryPartialResult, LogsTable, LogsQueryStatus
from azure.identity import DefaultAzureCredential
import datetime
from datetime import datetime, timedelta, timezone
import pandas as pd
import math
import json
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

start_time=datetime(2025, 4, 27, tzinfo=timezone.utc)
end_time=datetime(2025, 5, 1, tzinfo=timezone.utc)

df = pd.DataFrame()

try:
    response = client.query_workspace(
                workspace_id= 'c596239e-beda-4999-a79f-58e34d9881e4',
                query=vnet_logs,
                #timespan=None #timedelta(minutes=30)
                timespan=(start_time, end_time)
            )

    if response.status == LogsQueryStatus.SUCCESS:
        table = response.tables[0]
        df = pd.DataFrame(data=table.rows, columns=table.columns)

except HttpResponseError as err:
    print("something fatal happened")
    print(err)



In [0]:

# drop all logs where they are: Inbound-ExternalPublic and found in NTAIpDetails table
externalpublic_inbound_NTAIpDetails_match_indexes = df[(df['FlowDirection']=='Inbound') & (df['FlowType']=='ExternalPublic') & (df['SrcName'] != '')].index

tempdf = df.drop(externalpublic_inbound_NTAIpDetails_match_indexes)

# update SrcName by Subnet
                                               
tempdf['SrcName'] = tempdf.apply(lambda x: x['SrcSubnetName'] if x['SrcName']=='' else x['SrcName'], axis=1)

maindf = tempdf.drop_duplicates(subset=['SrcName', 'DestName'])

maindf.fillna('', inplace=True)

print(len(maindf.index))

print(maindf['FlowType'].unique())


maindf



126
['AzurePublic' 'ExternalPublic' 'UnknownPrivate' 'InterVNet' 'IntraVNet'
 'MaliciousFlow']


/root/.ipykernel/888/command-5477959247676144-1724500463:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf.fillna('', inplace=True)


,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,AzurePublic,Outbound,,tcp,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,928 Bytes,,,,,AzureCloud,0 Bytes,,,,13.65.254.147:22215,,southcentralus,
1,AzurePublic,Outbound,,tcp:https,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,managed vm in apim subnet,32.38 MB,,,,,AzureKeyVault,33.69 MB,,,,13.67.8.104:443,,southeastasia,
2,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,592.33 KB,,,,,AzureMonitor,253.69 KB,,,,13.71.196.56:1886,,westcentralus,
3,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,454.65 KB,,,,,AzureMonitor,68.3 KB,,,,13.71.196.56:1886,,westcentralus,
4,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,493.9 KB,,,,,AzureMonitor,253.49 KB,,,,13.71.196.56:1886,,westcentralus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.195.86.97,GatewayManager,28.84 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.4,28.11 KB,,southeastasia,,172.18.0.4:8443,,,
2049,AzurePublic,Inbound,,tcp:ov-nnm-websrv,,,,,40.90.185.46,AzureApiManagement,0 Bytes,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,managed vm in apim subnet,0 Bytes,,southeastasia,,172.16.2.5:3443,,,
2881,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.43.128.73,GatewayManager,27.07 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.5,28.38 KB,,southeastasia,,192.166.0.5:8444,,,
2882,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.43.128.73,GatewayManager,28.83 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.4,29.88 KB,,southeastasia,,192.166.0.4:8443,,,



### resolve vnet and subnet name of UnknownPrivate flow type 

In [0]:

from azure.identity import DefaultAzureCredential
from azure.mgmt.subscription import SubscriptionClient
from azure.mgmt.resourcegraph import ResourceGraphClient
from azure.mgmt.resourcegraph.models import *
import ipaddress

query = '''
resources
| where type == "microsoft.network/virtualnetworks"
| extend VNet = name
| mv-expand addressPrefixes = properties.addressSpace.addressPrefixes
| mv-expand subnet = properties.subnets
| project VNet, VNetAddressSpace = addressPrefixes, SubnetName=subnet.name, SubnetAddressPrefix=subnet.properties.addressPrefix
'''

azcred = DefaultAzureCredential()

# subscription ids
sub_client = SubscriptionClient(azcred)
subscription_list = sub_client.subscriptions.list()
subids = [item.subscription_id for item in subscription_list]

resourcegraph_client = ResourceGraphClient(
        credential=DefaultAzureCredential()
    )

    # Create Azure Resource Graph client and set options
query = QueryRequest(
            query=query,
            subscriptions=subids,
            options=QueryRequestOptions(
                result_format=ResultFormat.TABLE
            )
        )
query_response = resourcegraph_client.resources(query)


data_cols = [x['name'] for x in query_response.data['columns']]
data_row = query_response.data['rows']
vnet_subnet_names = pd.DataFrame(data_row, columns=data_cols)


# resolve vnet name and subnet name in maind from existing vnets
unknownprivate_df = maindf[maindf['FlowType'] == 'UnknownPrivate']

for index, ukprow in unknownprivate_df.iterrows():
    srcip = ukprow['SrcIp']
    destip = ukprow['DestIp']

    for index, vnet_subnet in vnet_subnet_names.iterrows():
        subnet_cid = vnet_subnet['SubnetAddressPrefix']
        
        try:

            if ipaddress.ip_address(srcip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['SrcVNet'] = vnet_subnet['VNet']
                unknownprivate_df['SrcSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['SrcName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

            if ipaddress.ip_address(destip) in ipaddress.ip_network(subnet_cid):
                unknownprivate_df['DestName'] = vnet_subnet['VNet']
                unknownprivate_df['DestSubnetName'] = vnet_subnet['SubnetName']
                unknownprivate_df['DestName'] = 'vm in ' +  unknownprivate_df["SrcSubnetName"]

        except Exception as e:
            continue


maindf.update(unknownprivate_df)

print('UnknownPrivate traffic updated SrcVNet, SrcSubnetName, DestVNet, DestSubnetName')


UnknownPrivate traffic updated SrcVNet, SrcSubnetName, DestVNet, DestSubnetName


/root/.ipykernel/888/command-6510015665464912-4159356528:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknownprivate_df['SrcVNet'] = vnet_subnet['VNet']
/root/.ipykernel/888/command-6510015665464912-4159356528:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknownprivate_df['SrcSubnetName'] = vnet_subnet['SubnetName']
/root/.ipykernel/888/command-6510015665464912-4159356528:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [0]:


maindf[maindf['FlowType'] == 'UnknownPrivate']



,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
469,UnknownPrivate,Inbound,,tcp:discard,,,vnet-cwhd-s2s-azure,AzureBastionSubnet,172.18.0.69,vm in AzureBastionSubnet,14.53 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,vms,vm-build-agent-win-server-1,110 Bytes,,,,10.0.1.75:9,,,



### create echarts nodes array

In [0]:
    # {
    #   name: 'CNF',
    #   id: 'CNF',
    #   category: 1,
    #   ip: '12.20.11.23',
    #   subnet: "some text",
    #   vnet: '123'
    # },



# create nodes csv for IntraVNet

src_nodes_final_df = pd.DataFrame()
src_nodes_final_df['id'] = maindf['SrcIp']
src_nodes_final_df['name'] = maindf['SrcName']
src_nodes_final_df['category'] = maindf['FlowType']
src_nodes_final_df['ip'] = maindf['SrcIp']
src_nodes_final_df['subnet'] = maindf['SrcSubnetName']
src_nodes_final_df['vnet'] = maindf['SrcVNet']
src_nodes_final_df['srcPIPLocation'] = maindf['SrcPIP_Location']


dest_nodes_final_df = pd.DataFrame()
dest_nodes_final_df['id'] = maindf['DestIp']
dest_nodes_final_df['name'] = maindf['DestName']
dest_nodes_final_df['category'] = maindf['FlowType']
dest_nodes_final_df['ip'] = maindf['DestIp']
dest_nodes_final_df['subnet'] = maindf['DestSubnetName']
dest_nodes_final_df['vnet'] = maindf['DestVNet']
src_nodes_final_df['destPIPLocation'] = maindf['DestPIP_Location']

# # #id,title,subtitle,mainstat,secondarystat,color
combined_list = [src_nodes_final_df, dest_nodes_final_df]

final_nodes_df = pd.concat(combined_list)

final_nodes_df = final_nodes_df.drop_duplicates(subset=['id'])

final_nodes_json_str =  final_nodes_df.to_json(orient='records')

final_nodes_json_obj = json.loads(final_nodes_json_str)


print(final_nodes_json_str)


[{"id":"192.166.0.5","name":"vnetgw-cwhd-onpremise-VM-192.166.0.5","category":"AzurePublic","ip":"192.166.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"172.18.0.5","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.5","category":"AzurePublic","ip":"172.18.0.5","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"192.166.0.4","name":"vnetgw-cwhd-onpremise-VM-192.166.0.4","category":"AzurePublic","ip":"192.166.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwh-s2s-onpremise","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"172.18.0.4","name":"vpngw-cwhd-s2s-azure-VM-172.18.0.4","category":"AzurePublic","ip":"172.18.0.4","subnet":"gatewaysubnet","vnet":"vnet-cwhd-s2s-azure","srcPIPLocation":"","destPIPLocation":"westcentralus"},{"id":"172.18.1.4","name":"vm-cwhd-s2s-azure","category":"AzurePublic","ip":"172.18.1.4","subnet":"vm","vnet":"vnet-cwhd-s2s-


### create echarts edges array

In [0]:

    # {
    #   source: 'CNF',
    #   target: 'SCPC',
    #   src_to_dest_data_size: '10 KB',
    #   dest_to_srct_data_size: '22 KB',
    #   flowType: 'Azure Public PaaS',
    #   label: {
    #     show: true,
    #     formatter: (params) => params.data.src_to_dest_data_size + '-> <- ' + params.data.dest_to_srct_data_size + '\n' + params.data.flowType,
    #     fontSize: 10
    #   },
    #   lineStyle: {
    #     curveness: 0.1
    #   }
    # }

# create edge

edges_df = pd.DataFrame()
edges_df['source'] = tempdf['SrcIp']
edges_df['target'] = tempdf['DestIp']
# edges_df['value'] = tempdf['SrcToDestDataSize'] + '-> <-' + tempdf['DestToSrcDataSize'] + '<div> flowtype: ' + tempdf['FlowType'] + '</div>' + '<div> protocol: ' +  tempdf['protocol'] + '/div>'
edges_df['src_to_dest_data_size'] = tempdf['SrcToDestDataSize']
edges_df['dest_to_srct_data_size'] = tempdf['DestToSrcDataSize']
edges_df['flowType'] = tempdf['FlowType']
edges_df['protocol'] = tempdf['Protocol']
edges_df['connectionType'] = tempdf['ConnectionType']

edges_df = edges_df.drop_duplicates(subset=['source', 'target'])

final_edges_json_str =  edges_df.to_json(orient='records')

final_edges_json_obj = json.loads(final_edges_json_str)

print(final_edges_json_str)


[{"source":"192.166.0.5","target":"13.71.196.56:1886","src_to_dest_data_size":"592.33 KB","dest_to_srct_data_size":"253.69 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"172.18.0.5","target":"13.71.196.56:1886","src_to_dest_data_size":"570.76 KB","dest_to_srct_data_size":"193.54 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"192.166.0.4","target":"13.71.196.56:1886","src_to_dest_data_size":"493.9 KB","dest_to_srct_data_size":"253.59 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"172.18.0.4","target":"13.71.196.56:443","src_to_dest_data_size":"1.37 MB","dest_to_srct_data_size":"489.8 KB","flowType":"AzurePublic","protocol":"tcp:https","connectionType":""},{"source":"172.18.0.4","target":"13.71.196.56:1886","src_to_dest_data_size":"503.11 KB","dest_to_srct_data_size":"273.88 KB","flowType":"AzurePublic","protocol":"tcp:leoip","connectionType":""},{"source":"172.18.0.5","target":

### categories result

In [0]:

# category

result = pd.DataFrame()
result['name'] = maindf['FlowType'].unique()

# result = list(result)
unique_categories =  result.to_json(orient='records')

print(unique_categories)

[{"name":"AzurePublic"},{"name":"ExternalPublic"},{"name":"UnknownPrivate"},{"name":"InterVNet"},{"name":"IntraVNet"},{"name":"MaliciousFlow"}]


### node and edges as single result

In [0]:
final_json = {
    "nodes": final_nodes_json_obj,
    "edges": final_edges_json_obj,
    "categories": unique_categories
}

json_str = json.dumps(final_json)

with open('nodes_edges.txt', "w") as f:
    f.write(json_str)


### unique source ip filter for grafana filter

In [0]:
maindf

,FlowType,FlowDirection,ConnectionType,Protocol,SrcSubscription,SrcRG,SrcVNet,SrcSubnetName,SrcIp,SrcName,SrcToDestDataSize,DestSubscription,DestRG,DestVNet,DestSubnetName,DestName,DestToSrcDataSize,SrcPIP_ThreatType,SrcPIP_Location,SrcPIP_Url,DestIp,DestPIP_ThreatType,DestPIP_Location,DestPIP_Url
0,AzurePublic,Outbound,,tcp,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.4,vnetgw-cwhd-onpremise-VM-192.166.0.4,928 Bytes,,,,,AzureCloud,0 Bytes,,,,13.65.254.147:22215,,southcentralus,
1,AzurePublic,Outbound,,tcp:https,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-ingress,apim,172.16.2.5,managed vm in apim subnet,31.5 MB,,,,,AzureKeyVault,33.24 MB,,,,13.67.8.104:443,,southeastasia,
2,AzurePublic,Outbound,,tcp:https,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-common,vnet-management,azurebastionsubnet,10.0.1.4,bastion-vm,7.32 MB,,,,,AzureKeyVault,10.33 MB,,,,13.67.8.104:443,,southeastasia,
3,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,192.166.0.5,vnetgw-cwhd-onpremise-VM-192.166.0.5,592.33 KB,,,,,AzureMonitor,253.69 KB,,,,13.71.196.56:1886,,westcentralus,
4,AzurePublic,Outbound,,tcp:leoip,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,172.18.0.5,vpngw-cwhd-s2s-azure-VM-172.18.0.5,556.98 KB,,,,,AzureMonitor,218.95 KB,,,,13.71.196.56:1886,,westcentralus,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,AzurePublic,Inbound,,tcp,,,,,40.87.200.0,AzureCloud,128 Bytes,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,vm,vm-cwhd-s2s-onpremise,0 Bytes,,japaneast,,192.166.1.4:22609,,,
4600,AzurePublic,Inbound,,tcp:pcsync-http,,,,,20.43.128.73,GatewayManager,27.07 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.5,28.38 KB,,southeastasia,,192.166.0.5:8444,,,
4601,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.43.128.73,GatewayManager,28.83 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwh-s2s-onpremise,gatewaysubnet,vnetgw-cwhd-onpremise-VM-192.166.0.4,26.98 KB,,southeastasia,,192.166.0.4:8443,,,
4602,AzurePublic,Inbound,,tcp:pcsync-https,,,,,20.195.86.98,GatewayManager,27.34 KB,c9061bc7-fa28-41d9-a783-2600b29c6e2f,rg-cwhd,vnet-cwhd-s2s-azure,gatewaysubnet,vpngw-cwhd-s2s-azure-VM-172.18.0.4,26.61 KB,,southeastasia,,172.18.0.4:8443,,,



### source ip filter

In [0]:
tempdf = maindf.drop_duplicates('SrcIp', keep='first')
tempdf = maindf[maindf['SrcIp'].notna()]
tempdf = maindf[maindf['SrcIp'].notnull()]

temp_without_vnet_df = tempdf[tempdf['SrcVNet'] == '']
temp_with_vnet_df = tempdf[(tempdf['SrcVNet'] != '') & (tempdf['SrcSubnetName'] != '')]

result_without_vnet_df = pd.DataFrame()
result_with_vnet_df = pd.DataFrame()


result_without_vnet_df['DisplayName'] = temp_without_vnet_df[['SrcName', 'SrcIp']].apply(lambda x: ' / '.join(x.dropna()) , axis=1 )
result_without_vnet_df['IP'] = temp_without_vnet_df['SrcIp']

result_with_vnet_df['DisplayName'] = temp_with_vnet_df[['SrcVNet', 'SrcSubnetName', 'SrcIp', 'SrcName']].apply(lambda x: ' / '.join(x.dropna()) , axis=1 )
result_with_vnet_df['IP'] = temp_with_vnet_df['SrcIp']

result = pd.concat([result_with_vnet_df, result_without_vnet_df])

result

,DisplayName,IP
0,vnet-cwh-s2s-onpremise / gatewaysubnet / 192.1...,192.166.0.4
1,vnet-ingress / apim / 172.16.2.5 / managed vm ...,172.16.2.5
2,vnet-management / azurebastionsubnet / 10.0.1....,10.0.1.4
3,vnet-cwh-s2s-onpremise / gatewaysubnet / 192.1...,192.166.0.5
4,vnet-cwhd-s2s-azure / gatewaysubnet / 172.18.0...,172.18.0.5
...,...,...
2112,GatewayManager / 20.43.128.73,20.43.128.73
3595,AzureApiManagement / 40.90.185.46,40.90.185.46
3596,GatewayManager / 20.195.86.98,20.195.86.98
3597,GatewayManager / 20.195.86.100,20.195.86.100


### dest ip filter

In [0]:
tempdf = maindf.drop_duplicates('DestIp', keep='first')
tempdf = maindf[(maindf['DestIp'].notna()) & maindf['DestIp'].notnull()]

temp_without_vnet_df = tempdf[tempdf['DestVNet'] == '']
temp_with_vnet_df = tempdf[(tempdf['DestVNet'] != '') & (tempdf['DestSubnetName'] != '')]

result_without_vnet_df = pd.DataFrame()
result_with_vnet_df = pd.DataFrame()


result_without_vnet_df['DisplayName'] = temp_without_vnet_df[['DestName', 'SrcIp']].apply(lambda x: ' / '.join(x.dropna()) , axis=1 )
result_without_vnet_df['IP'] = temp_without_vnet_df['DestIp']

result_with_vnet_df['DisplayName'] = temp_with_vnet_df[['DestVNet', 'DestSubnetName', 'DestIp', 'DestName']].apply(lambda x: ' / '.join(x.dropna()) , axis=1 )
result_with_vnet_df['IP'] = temp_with_vnet_df['DestIp']

result = pd.concat([result_with_vnet_df, result_without_vnet_df])

result

,DisplayName,IP
445,vnet-management / vms / 10.0.1.75:9 / vm-build...,10.0.1.75:9
446,vnet-management / vms / 10.0.1.75:80 / vm-buil...,10.0.1.75:80
447,vnet-cwh-s2s-onpremise / vm / 192.166.1.4:80 /...,192.166.1.4:80
449,vnet-cwhd-s2s-azure / gatewaysubnet / 172.18.0...,172.18.0.4:54321
450,vnet-cwh-s2s-onpremise / azurebastionsubnet / ...,192.166.0.68:5701
...,...,...
392,AzureEventHub / 192.166.0.4,20.195.81.84:5671
393,AzureEventHub / 172.18.0.4,20.195.81.84:5671
396,MicrosoftDefenderForEndpoint / 10.0.1.75,20.10.127.192:443
429,microsoft corporation / 172.16.2.5,52.123.252.3:443



### src vnet for filter

In [0]:
tempdf = maindf.drop_duplicates('SrcVNet', keep='first')
tempdf = tempdf[tempdf['SrcVNet'] != '']

result = pd.DataFrame()
result['DisplayName'] = tempdf['SrcVNet']
result


,DisplayName
0,vnet-cwh-s2s-onpremise
1,vnet-ingress
3,vnet-cwhd-s2s-azure
82,vnet-management
84,vnet-cwhd



### src subnet for filter

In [0]:

tempdf = maindf.copy()
tempdf['SrcSubnetName'] = maindf['SrcSubnetName'].str.lower()
tempdf = tempdf.drop_duplicates('SrcSubnetName', keep='first')
tempdf = tempdf[(tempdf['SrcVNet'] != '') & (tempdf['SrcSubnetName'] != '')]

temp_result = pd.DataFrame()
temp_result = tempdf[['SrcVNet', 'SrcSubnetName']]

result = pd.DataFrame()
result['DisplayName'] = temp_result.apply(lambda x: ' / '.join(x.dropna()), axis=1)
result['SubnetName'] = temp_result['SrcSubnetName']

result

,DisplayName,SubnetName
0,vnet-cwh-s2s-onpremise / gatewaysubnet,gatewaysubnet
1,vnet-ingress / apim,apim
18,vnet-cwhd-s2s-azure / azurebastionsubnet,azurebastionsubnet
82,vnet-management / vms,vms
83,vnet-cwhd-s2s-azure / vm,vm



### dest vnet for filter

In [0]:
tempdf = maindf.drop_duplicates('DestVNet', keep='first')
tempdf = tempdf[tempdf['DestVNet'] != '']

result = pd.DataFrame()
result['DisplayName'] = tempdf['DestVNet']
result

,DisplayName
469,vnet-management
471,vnet-cwh-s2s-onpremise
473,vnet-cwhd-s2s-azure
479,vnet-aks
498,vnet-cwhd
516,vnet-ingress



### dest subnet for filter

In [0]:

tempdf = maindf.copy()
tempdf['DestSubnetName'] = maindf['DestSubnetName'].str.lower()
tempdf = tempdf.drop_duplicates('DestSubnetName', keep='first')
tempdf = tempdf[(tempdf['DestVNet'] != '') & (tempdf['DestSubnetName'] != '')]

temp_result = pd.DataFrame()
temp_result = tempdf[['DestVNet', 'DestSubnetName']]

result = pd.DataFrame()
result['DisplayName'] = temp_result.apply(lambda x: ' / '.join(x.dropna()), axis=1)
result['SubnetName'] = temp_result['DestSubnetName']

result

,DisplayName,SubnetName
469,vnet-management / vms,vms
471,vnet-cwh-s2s-onpremise / vm,vm
473,vnet-cwhd-s2s-azure / gatewaysubnet,gatewaysubnet
474,vnet-cwh-s2s-onpremise / azurebastionsubnet,azurebastionsubnet
479,vnet-aks / appgateway,appgateway
498,vnet-cwhd / private-endpoints,private-endpoints
516,vnet-ingress / apim,apim
